## INFO 2950 Final Project Appendix
Larrisa Chen (lc949), Michelle Li (myl39), Christina Jin (cej65), Jade Eggleston (jce76)

# Data Cleaning


In [1]:
import numpy as np
import pandas as pd
import regex as re
import json

import ast
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

**(1)** We began the data cleaning process by tackling the listings datasets as they contained the most information we needed. Since many of the cells had missing values and were of type object, we used the notna function to pull out only the non-missing rows from each of our selected columns (host_about, host_response_time, host_response_rate, host_acceptance_rate, last_review, review_scores_rating, description, beds, bedrooms, bathrooms_text). So, any row that had any information missing in the above columns were removed.


In [3]:
# NEW YORK CLEANING

nyc_listings_df = pd.read_csv("listings/nyc_listings.csv")

In [4]:
# Pulling out the non-missing rows from our selected columns

nyc_listings_df = nyc_listings_df[nyc_listings_df["host_about"].notna()]
nyc_listings_df = nyc_listings_df[nyc_listings_df["host_response_time"].notna()]
nyc_listings_df = nyc_listings_df[nyc_listings_df["host_response_rate"].notna()]
nyc_listings_df = nyc_listings_df[nyc_listings_df["host_acceptance_rate"].notna()]
nyc_listings_df = nyc_listings_df[nyc_listings_df["last_review"].notna()]
nyc_listings_df = nyc_listings_df[nyc_listings_df["review_scores_rating"].notna()]
nyc_listings_df = nyc_listings_df[nyc_listings_df["description"].notna()]
nyc_listings_df = nyc_listings_df[nyc_listings_df["beds"].notna()]
nyc_listings_df = nyc_listings_df[nyc_listings_df["bedrooms"].notna()]
nyc_listings_df = nyc_listings_df[nyc_listings_df["bathrooms_text"].notna()]

**(2)** Many of the Airbnbs’ names, host descriptions, and listing information descriptions are showcased in various fonts and contain emoji symbols within them. However, we needed to make sure names and descriptions were uniform to make scraping for common words easier. So, we created the turn_into_string function to run through all the descriptions in each listing and remove any foreign characters or any emojis and any html inside the description. We also turned all the words inside the description to lowercase to standardize the words further Then, we created the is_string helper function to remove any empty strings after removing foreign characters. This removed the names and descriptions with special characters or fonts and ensured that all characters in descriptions or names were alphanumeric.

In [5]:
#is_string helper function

def is_string(x):   
    if type(x) != str:
        return False
    
    if x == " " or x == "":
        return False
        
    return True;

def turn_into_string(x):
    r = ""
    clean_string = re.compile("<.*?>|(?!(?<=[a-z])'[a-z])[^\w\s]") 

    for i in x.split("\n"):
        temp = re.sub(clean_string, ' ', i)
        r += temp.lower()
        
    return r

In [6]:
# Removed names and descriptions with special characters/fonts; ensure all characters/names are alphanumeric
nyc_listings_df["host_about"] = nyc_listings_df["host_about"].apply(turn_into_string)
nyc_listings_df = nyc_listings_df[nyc_listings_df["host_about"].apply(is_string)]

nyc_listings_df["name"] = nyc_listings_df["name"].apply(turn_into_string)
nyc_listings_df = nyc_listings_df[nyc_listings_df["name"].apply(is_string)]

nyc_listings_df["description"] = nyc_listings_df["description"].apply(turn_into_string)
nyc_listings_df = nyc_listings_df[nyc_listings_df["description"].apply(is_string)]
print(nyc_listings_df["description"])

0        renovated apt home in elevator building    the...
3         the space  beautiful  fully furnished and equ...
6         the space  charming standard queen room   the...
7        please don t expect the luxury here just a bas...
8        this room is up three flights of stairs   it's...
                               ...                        
39860    the place is ideal for a person or a couple ne...
39861    enjoy your nyc vacation stay in an elegant 3 b...
39864    enjoy breathtaking views of the nyc skyline ov...
39868    spacious apartment with great view of new york...
39869    modern industrial style apartment in jersey ci...
Name: description, Length: 11134, dtype: object


**(3)** For the following columns, we needed to convert the object types within our data into more exact data types that are valuable to our data analysis and exploration. Since host_response_rate and host_acceptance_rate were in a percentage format as a string, they were turned into type objects. Hence, we created the percent_to_float helper function to remove the percentage symbol and change the values into floats for these two columns. Additionally, the price column had a dollar sign in its values to represent dollar amounts, which were turned into type objects. Hence, we created the dollar_to_float helper function to remove the dollar symbol and change the values into floats for this column. 


In [7]:
# percent_to_float and dollar_to_float helper functions

def percent_to_float(x):
    return float(x.strip("%"))/100.0

def dollar_to_float(x):
    x = x.replace(",", "")
    x = x.replace("$", "")
    return float(x)

In [8]:
# Remove percentage symbol and change values into floats; remove dollar symbol and change values into floats

nyc_listings_df["host_response_rate"] = nyc_listings_df["host_response_rate"].apply(percent_to_float)
nyc_listings_df["host_acceptance_rate"] = nyc_listings_df["host_acceptance_rate"].apply(percent_to_float)
nyc_listings_df["price"] = nyc_listings_df["price"].apply(dollar_to_float)

**(4)** Moreover, the values under columns instant_bookable, host_identity_verified, host_has_profile_pic, and host_is_superhost were either “t” and “f”, which all became identified as type string or object. Hence, we created the str_to_bool helper function to convert the “t” and “f” into boolean expressions. We converted the “t” to True and “f” to False.


In [9]:
# str_to_bool helper function

def str_to_bool(x):
    if(x == "f"):
        return False
    else:
        return True

In [10]:
# Convert the "t" and "f" into boolean expressions

nyc_listings_df["instant_bookable"] = nyc_listings_df["instant_bookable"].apply(str_to_bool)
nyc_listings_df["host_identity_verified"] = nyc_listings_df["host_identity_verified"].apply(str_to_bool)
nyc_listings_df["host_has_profile_pic"] = nyc_listings_df["host_has_profile_pic"].apply(str_to_bool)
nyc_listings_df["host_is_superhost"] = nyc_listings_df["host_is_superhost"].apply(str_to_bool)

**(5)** There were categorical values under column host_response_time of “within an hour,” “within a few hours,” “within a day,” “a few days or more,” or “None”. To convert these values into an exact type, we decided to assign the categories to an int value within the range 0-4. The range would indicate the amount of time taken for the host response, with 0 indicating the shortest amount of time to 4 taking the longest amount of time. We created the host_response_time_to_int helper function to change the values “within an hour” to 0, “within a few hours” to 1, “within a day” to 2, “a few days or more” to 3, and “None” to 4. This would make it easier to numerically identify how long the host response takes and we can more easily quantify how long the host response takes.

In [11]:
# host_response_time_to_int helper function

def host_response_time_to_int(x):
    if(x == "within an hour"):
        return 0
    elif(x == "within a few hours"):
        return 1
    elif(x == "within a day"):
        return 2
    elif(x == "a few days or more"):
        return 3
    elif(x == "None"):
        return 4

In [12]:
# Change the categorical values into ints

nyc_listings_df["host_response_time"] = nyc_listings_df["host_response_time"].apply(host_response_time_to_int)

**(6)** Each description under the amenities columns was a single string, but we needed to turn them into lists of strings to scrape and identify common amenities among the listings. Hence, we created the amenities_to_list helper function, which uses a json library that automatically parses strings into a json format, and applied it to the amenities column.


In [13]:
# amenities_to_list helper function

def amenities_to_list(x):
    return json.loads(x)

In [14]:
# Turn into lists of strings to scrape and idnentify common ammenities among the listings

nyc_listings_df["amenities"] = nyc_listings_df["amenities"].apply(amenities_to_list)

**(7)** The values under columns host_since and last_review were type objects. We wanted to convert these values into datetime format to help with our visualizations, so we used the pandas to_datetime helper function to turn the values into type datetime. 

In [15]:
# Convert values into type datetime.

nyc_listings_df["host_since"] = pd.to_datetime(nyc_listings_df["host_since"])
nyc_listings_df["last_review"] = pd.to_datetime(nyc_listings_df["last_review"])

**(8)** Next, we wanted to create a privacy criteria of each listing dependent on whether users have to use a shared space. Although the room_type column labels listings as private rooms, there was still the possibility that the listing had a shared bathroom, making it not private. To resolve this, we decided that if bathroom_text had the word “shared” in it, the listing would be considered as a shared space.  In order to implement this, we first created the room_type_to_bool helper function to search through the room_type values and identify if the word “shared” was in it. We then created a new column called is_private_room to hold the room_type values after room_type_to_bool was applied to them. If the room_type value contained “shared,” then the room_type_to_bool function would return as True; if the room_type value did not contain “shared,” then the room_type_to_bool function would return as False. 


In [16]:
# room_type_to_bool helper function

def room_type_to_bool(x):
    if(x == "Shared room"):
        return False
    return True

In [17]:
# is_private_room column created to hold the room_type values

nyc_listings_df["is_private_room"] = nyc_listings_df["room_type"].apply(room_type_to_bool)

**(9)** Next, we completed the same process for bathrooms_text, creating the helper function bathrooms_text_to_bool to search through the bathrooms_text values and identify if “shared” was in it. We made a new column called is_private_bath to hold the resulting True/False values.

In [18]:
# bathrooms_text_to_bool helper function

def bathrooms_text_to_bool(x):
    if(str(x).find("shared") != -1 or str(x).find("Shared") != -1):
        return False
    return True

In [19]:
# is_private_bath column created to hold the True/False values 

nyc_listings_df["is_private_bath"] = nyc_listings_df["bathrooms_text"].apply(bathrooms_text_to_bool)

**(10)** Finally, we created a new column called is_private_overall to determine whether the listing is private based on whether both values in the is_private_room and is_private bathroom columns are True.

In [20]:
# is_private_overall column created

nyc_listings_df["is_private_overall"] = nyc_listings_df["is_private_room"] & nyc_listings_df["is_private_bath"]

**(11)** In addition, we wanted to turn the bathrooms_text column values (1 bathroom, 1.5 bathrooms, or half-bath) into type ints to make visualizations easier. We created the bathrooms_text_to_int helper function to parse through bathrooms_text and identify the “half-bath” values and return it as 0.5. If the value did not include “half-bath,” the function would just use regex to return only the number. The resulting values from the bathrooms_text_to_int helper function were then added to a new column called baths.

In [21]:
# bathroom_text_to_float helper function

def bathroom_text_to_float(x):
    if(x.find("half-bath") != -1 or x.find("Half-bath") != -1):
        return 0.5
    elif(re.search(r"d+\.\d+", x) != None):
        return float(re.search(r"d+\.\d+", x).group())
    return int(re.search(r"\d+", x).group())

In [22]:
# Turn column values into type int

nyc_listings_df["baths"] = nyc_listings_df["bathrooms_text"].apply(bathroom_text_to_float)

In [23]:
print(nyc_listings_df.dtypes)
print(nyc_listings_df.shape)
print(nyc_listings_df)

id                               float64
name                              object
description                       object
host_since                datetime64[ns]
host_about                        object
host_response_time                 int64
host_response_rate               float64
host_acceptance_rate             float64
host_is_superhost                   bool
host_listings_count              float64
host_has_profile_pic                bool
host_identity_verified              bool
neighbourhood_cleansed            object
room_type                         object
accommodates                       int64
bathrooms_text                    object
bedrooms                         float64
beds                             float64
amenities                         object
price                            float64
minimum_nights                     int64
maximum_nights                     int64
number_of_reviews                  int64
last_review               datetime64[ns]
review_scores_ra

**(12)** After finishing the cleaning in the listings data sets, we began cleaning the calendar data sets. We used the drop function to remove the price and adjusted_price columns since we already had the same information in our listings data sets. 


In [24]:
# Calendar datasets

nyc_calendar_df = pd.read_csv("calendars/nyc_calendar.csv")

In [25]:
# Remove two columns (redundant information)

nyc_calendar_df = nyc_calendar_df.drop("price", axis=1)
nyc_calendar_df = nyc_calendar_df.drop("adjusted_price", axis=1)

**(13)** Additionally, we converted the date column values to type datetime and the available column values to boolean using the str_to_bool helper function.

In [26]:
# Convert date column values into type datetime and available column values into boolean

nyc_calendar_df["date"] = pd.to_datetime(nyc_calendar_df["date"])
nyc_calendar_df["available"] = nyc_calendar_df["available"].apply(str_to_bool)

In [27]:
print(nyc_calendar_df)
print(nyc_calendar_df.dtypes)

                  listing_id       date  available  minimum_nights  \
0                       2539 2022-09-07      False            30.0   
1                       2539 2022-09-08      False            30.0   
2                       2539 2022-09-09      False            30.0   
3                       2539 2022-09-10      False            30.0   
4                       2539 2022-09-11      False            30.0   
...                      ...        ...        ...             ...   
14551457  709854858248316418 2023-09-02       True            30.0   
14551458  709854858248316418 2023-09-03       True            30.0   
14551459  709854858248316418 2023-09-04       True            30.0   
14551460  709854858248316418 2023-09-05       True            30.0   
14551461  709854858248316418 2023-09-06       True            30.0   

          maximum_nights  
0                  730.0  
1                  730.0  
2                  730.0  
3                  730.0  
4                  730.0

In [28]:
%sql nyc_bookings_df << (SELECT listing_id, COUNT(available) AS days_booked FROM nyc_calendar_df WHERE available = False GROUP BY listing_id)
%sql nyc_combined_df << (SELECT * FROM nyc_listings_df LEFT JOIN nyc_bookings_df ON nyc_listings_df.id = nyc_bookings_df.listing_id)
%sql nyc_listings_df << SELECT * FROM nyc_listings_df ORDER BY price 
%sql nyc_combined_df << SELECT * FROM nyc_combined_df ORDER BY price 

Returning data to local variable nyc_bookings_df
Returning data to local variable nyc_combined_df
Returning data to local variable nyc_listings_df
Returning data to local variable nyc_combined_df


In [29]:
nyc_combined_df = nyc_combined_df.drop(columns = 'listing_id')
nyc_listings_df = nyc_listings_df.drop([11094, 11095, 11096])
nyc_combined_df = nyc_combined_df.drop([11094, 11095, 11096])

**(14)** Because the data sets for each city were similar in data format in terms of the columns and the types we wanted the columns to be, we used the same process for data cleaning for every city. (NYC done)

#### NYC Amenities Scraping

Here we are counting the frequencies of every amenity listed in every NYC listing and creating a dataframe to be able to get the most frequent amenities listed.

**(1)** We wanted to create a list that included all the amenities offered in all the listings within one city. So, we created a for loop that would pull each amenity from the listings into a single list. 

In [30]:
nyc_amenities_df = pd.DataFrame(nyc_combined_df['amenities'])
nyc_amenities_list_accum = []
df_len = len(nyc_amenities_df.index)
for i in range(0,df_len):
    x = ast.literal_eval(nyc_amenities_df.iloc[i, 0])
    nyc_amenities_list_accum.append(x)

**(2)** We wanted to match each amenity with the frequency it appeared in the list we just created. Hence, we made another for loop to count the frequency of each amenity that would output a dictionary.

In [31]:
nyc_amenities_frequency={}
for list in nyc_amenities_list_accum:
    for item in list:
        if item in nyc_amenities_frequency:
            nyc_amenities_frequency[item] += 1
        else:
            nyc_amenities_frequency[item] = 1

**(3)** Here, we pulled the dictionary into the nyc_amenities_df dataframe. We then used the pandas melt function to reshape the dataframe so that the columns would become variable and could include all amenities and the count of each amenity.

In [112]:
nyc_amenities_df = pd.DataFrame(nyc_amenities_frequency, index=[0])
nyc_amenities_list = nyc_amenities_df.columns

In [33]:
nyc_amenities_df = pd.melt(nyc_amenities_df, value_vars=nyc_amenities_list, value_name='nyc_count')

**(4)** Finally, we used SQL to pull out the 50 most frequently listed amenity.

In [34]:
%sql nyc_amenities_df << SELECT * FROM nyc_amenities_df ORDER BY nyc_count DESC FETCH FIRST 50 ROWS ONLY;

Returning data to local variable nyc_amenities_df


**(5)** We used the same process for cleaning the amenities for every city. (NYC done)

## Boston Cleaning

In [35]:
# BOSTON CLEANING

bos_listings_df = pd.read_csv("listings/bos_listings.csv")

bos_listings_df = bos_listings_df[bos_listings_df["host_about"].notna()]
bos_listings_df = bos_listings_df[bos_listings_df["host_response_time"].notna()]
bos_listings_df = bos_listings_df[bos_listings_df["host_response_rate"].notna()]
bos_listings_df = bos_listings_df[bos_listings_df["host_acceptance_rate"].notna()]
bos_listings_df = bos_listings_df[bos_listings_df["last_review"].notna()]
bos_listings_df = bos_listings_df[bos_listings_df["review_scores_rating"].notna()]
bos_listings_df = bos_listings_df[bos_listings_df["description"].notna()]
bos_listings_df = bos_listings_df[bos_listings_df["beds"].notna()]
bos_listings_df = bos_listings_df[bos_listings_df["bedrooms"].notna()]
bos_listings_df = bos_listings_df[bos_listings_df["bathrooms_text"].notna()]

bos_listings_df = bos_listings_df[bos_listings_df["host_about"].apply(is_string)]
bos_listings_df = bos_listings_df[bos_listings_df["name"].apply(is_string)]
bos_listings_df = bos_listings_df[bos_listings_df["description"].apply(is_string)]

bos_listings_df["host_response_rate"] = bos_listings_df["host_response_rate"].apply(percent_to_float)
bos_listings_df["host_acceptance_rate"] = bos_listings_df["host_acceptance_rate"].apply(percent_to_float)
bos_listings_df["price"] = bos_listings_df["price"].apply(dollar_to_float)
bos_listings_df["instant_bookable"] = bos_listings_df["instant_bookable"].apply(str_to_bool)
bos_listings_df["host_identity_verified"] = bos_listings_df["host_identity_verified"].apply(str_to_bool)
bos_listings_df["host_has_profile_pic"] = bos_listings_df["host_has_profile_pic"].apply(str_to_bool)
bos_listings_df["host_is_superhost"] = bos_listings_df["host_is_superhost"].apply(str_to_bool)
bos_listings_df["host_response_time"] = bos_listings_df["host_response_time"].apply(host_response_time_to_int)
bos_listings_df["amenities"] = bos_listings_df["amenities"].apply(amenities_to_list)

bos_listings_df["is_private_room"] = bos_listings_df["room_type"].apply(room_type_to_bool)
bos_listings_df["is_private_bath"] = bos_listings_df["bathrooms_text"].apply(bathrooms_text_to_bool)
bos_listings_df["baths"] = bos_listings_df["bathrooms_text"].apply(bathroom_text_to_float)
bos_listings_df["is_private_overall"] = bos_listings_df["is_private_room"] & bos_listings_df["is_private_bath"]

bos_listings_df["host_since"] = pd.to_datetime(bos_listings_df["host_since"])
bos_listings_df["last_review"] = pd.to_datetime(bos_listings_df["last_review"])

print(bos_listings_df.dtypes)
print(bos_listings_df.shape)
print(bos_listings_df)

bos_calendar_df = pd.read_csv("calendars/bos_calendar.csv")
bos_calendar_df = bos_calendar_df.drop("price", axis=1)
bos_calendar_df = bos_calendar_df.drop("adjusted_price", axis=1)
bos_calendar_df["date"] = pd.to_datetime(bos_calendar_df["date"])
bos_calendar_df["available"] = bos_calendar_df["available"].apply(str_to_bool)

print(bos_calendar_df)
print(bos_calendar_df.dtypes)

id                               float64
name                              object
description                       object
host_since                datetime64[ns]
host_about                        object
host_response_time                 int64
host_response_rate               float64
host_acceptance_rate             float64
host_is_superhost                   bool
host_listings_count                int64
host_has_profile_pic                bool
host_identity_verified              bool
neighbourhood_cleansed            object
room_type                         object
accommodates                       int64
bathrooms_text                    object
bedrooms                         float64
beds                             float64
amenities                         object
price                            float64
minimum_nights                     int64
maximum_nights                     int64
number_of_reviews                  int64
last_review               datetime64[ns]
review_scores_ra

In [36]:
%sql bos_bookings_df << (SELECT listing_id, COUNT(available) AS days_booked FROM bos_calendar_df WHERE available = False GROUP BY listing_id)
%sql bos_combined_df << (SELECT * FROM bos_listings_df LEFT JOIN bos_bookings_df ON bos_listings_df.id = bos_bookings_df.listing_id)
%sql bos_listings_df << SELECT * FROM bos_listings_df ORDER BY price 
%sql bos_combined_df << SELECT * FROM bos_combined_df ORDER BY price 

Returning data to local variable bos_bookings_df
Returning data to local variable bos_combined_df
Returning data to local variable bos_listings_df
Returning data to local variable bos_combined_df


In [37]:
bos_combined_df = bos_combined_df.drop(columns = 'listing_id')
bos_listings_df = bos_listings_df.drop([1982, 1983])
bos_combined_df = bos_combined_df.drop([1982, 1983])

#### BOS Amenity Scraping

In [38]:
bos_amenities_df = pd.DataFrame(bos_combined_df['amenities'])
bos_amenities_list_accum = []
df_len = len(bos_amenities_df.index)
for i in range(0,df_len):
    x = ast.literal_eval(bos_amenities_df.iloc[i, 0])
    bos_amenities_list_accum.append(x)

In [39]:
bos_amenities_frequency={}
for list in bos_amenities_list_accum:
    for item in list:
        if item in bos_amenities_frequency:
            bos_amenities_frequency[item] += 1
        else:
            bos_amenities_frequency[item] = 1

In [40]:
bos_amenities_df = pd.DataFrame(bos_amenities_frequency, index=[0])
bos_amenities_list = bos_amenities_df.columns

In [41]:
bos_amenities_df = pd.melt(bos_amenities_df, value_vars=bos_amenities_list, value_name='bos_count')

In [42]:
%sql bos_amenities_df << SELECT * FROM bos_amenities_df ORDER BY bos_count DESC FETCH FIRST 50 ROWS ONLY;

Returning data to local variable bos_amenities_df


In [43]:
bos_amenities_df = bos_amenities_df.rename(columns={'variable': 'variable_1'})

## Chicago Cleaning

In [44]:
# CHICAGO CLEANING

chi_listings_df = pd.read_csv("listings/chi_listings.csv")

chi_listings_df = chi_listings_df[chi_listings_df["host_about"].notna()]
chi_listings_df = chi_listings_df[chi_listings_df["host_response_time"].notna()]
chi_listings_df = chi_listings_df[chi_listings_df["host_response_rate"].notna()]
chi_listings_df = chi_listings_df[chi_listings_df["host_acceptance_rate"].notna()]
chi_listings_df = chi_listings_df[chi_listings_df["last_review"].notna()]
chi_listings_df = chi_listings_df[chi_listings_df["review_scores_rating"].notna()]
chi_listings_df = chi_listings_df[chi_listings_df["description"].notna()]
chi_listings_df = chi_listings_df[chi_listings_df["beds"].notna()]
chi_listings_df = chi_listings_df[chi_listings_df["bedrooms"].notna()]
chi_listings_df = chi_listings_df[chi_listings_df["bathrooms_text"].notna()]

chi_listings_df = chi_listings_df[chi_listings_df["host_about"].apply(is_string)]
chi_listings_df = chi_listings_df[chi_listings_df["name"].apply(is_string)]
chi_listings_df = chi_listings_df[chi_listings_df["description"].apply(is_string)]

chi_listings_df["host_response_rate"] = chi_listings_df["host_response_rate"].apply(percent_to_float)
chi_listings_df["host_acceptance_rate"] = chi_listings_df["host_acceptance_rate"].apply(percent_to_float)
chi_listings_df["price"] = chi_listings_df["price"].apply(dollar_to_float)
chi_listings_df["instant_bookable"] = chi_listings_df["instant_bookable"].apply(str_to_bool)
chi_listings_df["host_identity_verified"] = chi_listings_df["host_identity_verified"].apply(str_to_bool)
chi_listings_df["host_has_profile_pic"] = chi_listings_df["host_has_profile_pic"].apply(str_to_bool)
chi_listings_df["host_is_superhost"] = chi_listings_df["host_is_superhost"].apply(str_to_bool)
chi_listings_df["host_response_time"] = chi_listings_df["host_response_time"].apply(host_response_time_to_int)
chi_listings_df["amenities"] = chi_listings_df["amenities"].apply(amenities_to_list)

chi_listings_df["is_private_room"] = chi_listings_df["room_type"].apply(room_type_to_bool)
chi_listings_df["is_private_bath"] = chi_listings_df["bathrooms_text"].apply(bathrooms_text_to_bool)
chi_listings_df["baths"] = chi_listings_df["bathrooms_text"].apply(bathroom_text_to_float)
chi_listings_df["is_private_overall"] = chi_listings_df["is_private_room"] & chi_listings_df["is_private_bath"]

chi_listings_df["host_since"] = pd.to_datetime(chi_listings_df["host_since"])
chi_listings_df["last_review"] = pd.to_datetime(chi_listings_df["last_review"])

print(chi_listings_df.dtypes)
print(chi_listings_df.shape)
print(chi_listings_df)

chi_calendar_df = pd.read_csv("calendars/chi_calendar.csv")
chi_calendar_df = chi_calendar_df.drop("price", axis=1)
chi_calendar_df = chi_calendar_df.drop("adjusted_price", axis=1)
chi_calendar_df["date"] = pd.to_datetime(chi_calendar_df["date"])
chi_calendar_df["available"] = chi_calendar_df["available"].apply(str_to_bool)

print(chi_calendar_df)
print(chi_calendar_df.dtypes)

id                               float64
name                              object
description                       object
host_since                datetime64[ns]
host_about                        object
host_response_time                 int64
host_response_rate               float64
host_acceptance_rate             float64
host_is_superhost                   bool
host_listings_count                int64
host_has_profile_pic                bool
host_identity_verified              bool
neighbourhood_cleansed            object
room_type                         object
accommodates                       int64
bathrooms_text                    object
bedrooms                         float64
beds                             float64
amenities                         object
price                            float64
minimum_nights                     int64
maximum_nights                     int64
number_of_reviews                  int64
last_review               datetime64[ns]
review_scores_ra

In [45]:
%sql chi_bookings_df << (SELECT listing_id, COUNT(available) AS days_booked FROM chi_calendar_df WHERE available = False GROUP BY listing_id)
%sql chi_combined_df << (SELECT * FROM chi_listings_df LEFT JOIN chi_bookings_df ON chi_listings_df.id = chi_bookings_df.listing_id)
%sql chi_listings_df << SELECT * FROM chi_listings_df ORDER BY price 
%sql chi_combined_df << SELECT * FROM chi_combined_df ORDER BY price 

Returning data to local variable chi_bookings_df
Returning data to local variable chi_combined_df
Returning data to local variable chi_listings_df
Returning data to local variable chi_combined_df


In [46]:
chi_combined_df = chi_combined_df.drop(columns = 'listing_id')
chi_listings_df = chi_listings_df.drop([3384])
chi_combined_df = chi_combined_df.drop([3384])

#### CHI Amenity Scraping

In [47]:
chi_amenities_df = pd.DataFrame(chi_combined_df['amenities'])
chi_amenities_list_accum = []
df_len = len(chi_amenities_df.index)
for i in range(0,df_len):
    x = ast.literal_eval(chi_amenities_df.iloc[i, 0])
    chi_amenities_list_accum.append(x)

In [48]:
chi_amenities_frequency={}
for list in chi_amenities_list_accum:
    for item in list:
        if item in chi_amenities_frequency:
            chi_amenities_frequency[item] += 1
        else:
            chi_amenities_frequency[item] = 1

In [49]:
chi_amenities_df = pd.DataFrame(chi_amenities_frequency, index=[0])
chi_amenities_list = chi_amenities_df.columns

In [50]:
chi_amenities_df = pd.melt(chi_amenities_df, value_vars=chi_amenities_list, value_name='chi_count')

In [51]:
%sql chi_amenities_df << SELECT * FROM chi_amenities_df ORDER BY chi_count DESC FETCH FIRST 50 ROWS ONLY;

Returning data to local variable chi_amenities_df


In [52]:
chi_amenities_df = chi_amenities_df.rename(columns={'variable': 'variable_1'})

## Washington DC Cleaning

In [53]:
# WASHINGTON DC CLEANING

dc_listings_df = pd.read_csv("listings/dc_listings.csv")

dc_listings_df = dc_listings_df[dc_listings_df["host_about"].notna()]
dc_listings_df = dc_listings_df[dc_listings_df["host_response_time"].notna()]
dc_listings_df = dc_listings_df[dc_listings_df["host_response_rate"].notna()]
dc_listings_df = dc_listings_df[dc_listings_df["host_acceptance_rate"].notna()]
dc_listings_df = dc_listings_df[dc_listings_df["last_review"].notna()]
dc_listings_df = dc_listings_df[dc_listings_df["review_scores_rating"].notna()]
dc_listings_df = dc_listings_df[dc_listings_df["description"].notna()]
dc_listings_df = dc_listings_df[dc_listings_df["beds"].notna()]
dc_listings_df = dc_listings_df[dc_listings_df["bedrooms"].notna()]
dc_listings_df = dc_listings_df[dc_listings_df["bathrooms_text"].notna()]

dc_listings_df = dc_listings_df[dc_listings_df["host_about"].apply(is_string)]
dc_listings_df = dc_listings_df[dc_listings_df["name"].apply(is_string)]
dc_listings_df = dc_listings_df[dc_listings_df["description"].apply(is_string)]

dc_listings_df["host_response_rate"] = dc_listings_df["host_response_rate"].apply(percent_to_float)
dc_listings_df["host_acceptance_rate"] = dc_listings_df["host_acceptance_rate"].apply(percent_to_float)
dc_listings_df["price"] = dc_listings_df["price"].apply(dollar_to_float)
dc_listings_df["instant_bookable"] = dc_listings_df["instant_bookable"].apply(str_to_bool)
dc_listings_df["host_identity_verified"] = dc_listings_df["host_identity_verified"].apply(str_to_bool)
dc_listings_df["host_has_profile_pic"] = dc_listings_df["host_has_profile_pic"].apply(str_to_bool)
dc_listings_df["host_is_superhost"] = dc_listings_df["host_is_superhost"].apply(str_to_bool)
dc_listings_df["host_response_time"] = dc_listings_df["host_response_time"].apply(host_response_time_to_int)
dc_listings_df["amenities"] = dc_listings_df["amenities"].apply(amenities_to_list)

dc_listings_df["is_private_room"] = dc_listings_df["room_type"].apply(room_type_to_bool)
dc_listings_df["is_private_bath"] = dc_listings_df["bathrooms_text"].apply(bathrooms_text_to_bool)
dc_listings_df["baths"] = dc_listings_df["bathrooms_text"].apply(bathroom_text_to_float)
dc_listings_df["is_private_overall"] = dc_listings_df["is_private_room"] & dc_listings_df["is_private_bath"]

dc_listings_df["host_since"] = pd.to_datetime(dc_listings_df["host_since"])
dc_listings_df["last_review"] = pd.to_datetime(dc_listings_df["last_review"])

print(dc_listings_df.dtypes)
print(dc_listings_df.shape)
print(dc_listings_df)

dc_calendar_df = pd.read_csv("calendars/dc_calendar.csv")
dc_calendar_df = dc_calendar_df.drop("price", axis=1)
dc_calendar_df = dc_calendar_df.drop("adjusted_price", axis=1)
dc_calendar_df["date"] = pd.to_datetime(dc_calendar_df["date"])
dc_calendar_df["available"] = dc_calendar_df["available"].apply(str_to_bool)

print(dc_calendar_df)
print(dc_calendar_df.dtypes)

id                               float64
name                              object
description                       object
host_since                datetime64[ns]
host_about                        object
host_response_time                 int64
host_response_rate               float64
host_acceptance_rate             float64
host_is_superhost                   bool
host_listings_count              float64
host_has_profile_pic                bool
host_identity_verified              bool
neighbourhood_cleansed            object
room_type                         object
accommodates                       int64
bathrooms_text                    object
bedrooms                         float64
beds                             float64
amenities                         object
price                            float64
minimum_nights                     int64
maximum_nights                     int64
number_of_reviews                  int64
last_review               datetime64[ns]
review_scores_ra

In [54]:
%sql dc_bookings_df << (SELECT listing_id, COUNT(available) AS days_booked FROM dc_calendar_df WHERE available = False GROUP BY listing_id)
%sql dc_combined_df << (SELECT * FROM dc_listings_df LEFT JOIN dc_bookings_df ON dc_listings_df.id = dc_bookings_df.listing_id)
%sql dc_listings_df << SELECT * FROM dc_listings_df ORDER BY price 
%sql dc_combined_df << SELECT * FROM dc_combined_df ORDER BY price 

Returning data to local variable dc_bookings_df
Returning data to local variable dc_combined_df
Returning data to local variable dc_listings_df
Returning data to local variable dc_combined_df


In [55]:
dc_combined_df = dc_combined_df.drop(columns = 'listing_id')
dc_listings_df = dc_listings_df.drop([2493, 2494])
dc_combined_df = dc_combined_df.drop([2493, 2494])

#### DC Amenity Scraping

In [56]:
dc_amenities_df = pd.DataFrame(dc_combined_df['amenities'])
dc_amenities_list_accum = []
df_len = len(dc_amenities_df.index)
for i in range(0,df_len):
    x = ast.literal_eval(dc_amenities_df.iloc[i, 0])
    dc_amenities_list_accum.append(x)

In [57]:
dc_amenities_frequency={}
for list in dc_amenities_list_accum:
    for item in list:
        if item in dc_amenities_frequency:
            dc_amenities_frequency[item] += 1
        else:
            dc_amenities_frequency[item] = 1

In [58]:
dc_amenities_df = pd.DataFrame(dc_amenities_frequency, index=[0])
dc_amenities_list = dc_amenities_df.columns

In [59]:
dc_amenities_df = pd.melt(dc_amenities_df, value_vars=dc_amenities_list, value_name='dc_count')

In [60]:
%sql dc_amenities_df << SELECT * FROM dc_amenities_df ORDER BY dc_count DESC FETCH FIRST 50 ROWS ONLY;

Returning data to local variable dc_amenities_df


In [61]:
dc_amenities_df = dc_amenities_df.rename(columns={'variable': 'variable_1'})

## Dallas Cleaning

In [62]:
# DALLAS CLEANING

dal_listings_df = pd.read_csv("listings/dal_listings.csv")

dal_listings_df = dal_listings_df[dal_listings_df["host_about"].notna()]
dal_listings_df = dal_listings_df[dal_listings_df["host_response_time"].notna()]
dal_listings_df = dal_listings_df[dal_listings_df["host_response_rate"].notna()]
dal_listings_df = dal_listings_df[dal_listings_df["host_acceptance_rate"].notna()]
dal_listings_df = dal_listings_df[dal_listings_df["last_review"].notna()]
dal_listings_df = dal_listings_df[dal_listings_df["review_scores_rating"].notna()]
dal_listings_df = dal_listings_df[dal_listings_df["description"].notna()]
dal_listings_df = dal_listings_df[dal_listings_df["beds"].notna()]
dal_listings_df = dal_listings_df[dal_listings_df["bedrooms"].notna()]
dal_listings_df = dal_listings_df[dal_listings_df["bathrooms_text"].notna()]

dal_listings_df = dal_listings_df[dal_listings_df["host_about"].apply(is_string)]
dal_listings_df = dal_listings_df[dal_listings_df["name"].apply(is_string)]
dal_listings_df = dal_listings_df[dal_listings_df["description"].apply(is_string)]

dal_listings_df["host_response_rate"] = dal_listings_df["host_response_rate"].apply(percent_to_float)
dal_listings_df["host_acceptance_rate"] = dal_listings_df["host_acceptance_rate"].apply(percent_to_float)
dal_listings_df["price"] = dal_listings_df["price"].apply(dollar_to_float)
dal_listings_df["instant_bookable"] = dal_listings_df["instant_bookable"].apply(str_to_bool)
dal_listings_df["host_identity_verified"] = dal_listings_df["host_identity_verified"].apply(str_to_bool)
dal_listings_df["host_has_profile_pic"] = dal_listings_df["host_has_profile_pic"].apply(str_to_bool)
dal_listings_df["host_is_superhost"] = dal_listings_df["host_is_superhost"].apply(str_to_bool)
dal_listings_df["host_response_time"] = dal_listings_df["host_response_time"].apply(host_response_time_to_int)
dal_listings_df["amenities"] = dal_listings_df["amenities"].apply(amenities_to_list)

dal_listings_df["is_private_room"] = dal_listings_df["room_type"].apply(room_type_to_bool)
dal_listings_df["is_private_bath"] = dal_listings_df["bathrooms_text"].apply(bathrooms_text_to_bool)
dal_listings_df["baths"] = dal_listings_df["bathrooms_text"].apply(bathroom_text_to_float)
dal_listings_df["is_private_overall"] = dal_listings_df["is_private_room"] & dal_listings_df["is_private_bath"]

dal_listings_df["host_since"] = pd.to_datetime(dal_listings_df["host_since"])
dal_listings_df["last_review"] = pd.to_datetime(dal_listings_df["last_review"])

print(dal_listings_df.dtypes)
print(dal_listings_df.shape)
print(dal_listings_df)

dal_calendar_df = pd.read_csv("calendars/dal_calendar.csv")
dal_calendar_df = dal_calendar_df.drop("price", axis=1)
dal_calendar_df = dal_calendar_df.drop("adjusted_price", axis=1)
dal_calendar_df["date"] = pd.to_datetime(dal_calendar_df["date"])
dal_calendar_df["available"] = dal_calendar_df["available"].apply(str_to_bool)

print(dal_calendar_df)
print(dal_calendar_df.dtypes)

id                               float64
name                              object
description                       object
host_since                datetime64[ns]
host_about                        object
host_response_time                 int64
host_response_rate               float64
host_acceptance_rate             float64
host_is_superhost                   bool
host_listings_count                int64
host_has_profile_pic                bool
host_identity_verified              bool
neighbourhood_cleansed            object
room_type                         object
accommodates                       int64
bathrooms_text                    object
bedrooms                         float64
beds                             float64
amenities                         object
price                            float64
minimum_nights                     int64
maximum_nights                     int64
number_of_reviews                  int64
last_review               datetime64[ns]
review_scores_ra

In [63]:
%sql dal_bookings_df << (SELECT listing_id, COUNT(available) AS days_booked FROM dal_calendar_df WHERE available = False GROUP BY listing_id)
%sql dal_combined_df << (SELECT * FROM dal_listings_df LEFT JOIN dal_bookings_df ON dal_listings_df.id = dal_bookings_df.listing_id)
%sql dal_listings_df << SELECT * FROM dal_listings_df ORDER BY price 
%sql dal_combined_df << SELECT * FROM dal_combined_df ORDER BY price 

Returning data to local variable dal_bookings_df
Returning data to local variable dal_combined_df
Returning data to local variable dal_listings_df
Returning data to local variable dal_combined_df


In [64]:
dal_combined_df = dal_combined_df.drop(columns = 'listing_id')
dal_listings_df = dal_listings_df.drop([2795])
dal_combined_df = dal_combined_df.drop([2795])

#### DAL Amenity Scraping


In [65]:
dal_amenities_df = pd.DataFrame(dal_combined_df['amenities'])
dal_amenities_list_accum = []
df_len = len(dal_amenities_df.index)
for i in range(0,df_len):
    x = ast.literal_eval(dal_amenities_df.iloc[i, 0])
    dal_amenities_list_accum.append(x)

In [66]:
dal_amenities_frequency={}
for list in dal_amenities_list_accum:
    for item in list:
        if item in dal_amenities_frequency:
            dal_amenities_frequency[item] += 1
        else:
            dal_amenities_frequency[item] = 1

In [67]:
dal_amenities_df = pd.DataFrame(dal_amenities_frequency, index=[0])
dal_amenities_list = dal_amenities_df.columns

In [68]:
dal_amenities_df = pd.melt(dal_amenities_df, value_vars=dal_amenities_list, value_name='dal_count')

In [69]:
%sql dal_amenities_df << SELECT * FROM dal_amenities_df ORDER BY dal_count DESC FETCH FIRST 50 ROWS ONLY;

Returning data to local variable dal_amenities_df


In [70]:
dal_amenities_df = dal_amenities_df.rename(columns={'variable': 'variable_1'})

## Las Vegas Cleaning

In [71]:
# LAS VEGAS CLEANING

lv_listings_df = pd.read_csv("listings/lv_listings.csv")

lv_listings_df = lv_listings_df[lv_listings_df["host_about"].notna()]
lv_listings_df = lv_listings_df[lv_listings_df["host_response_time"].notna()]
lv_listings_df = lv_listings_df[lv_listings_df["host_response_rate"].notna()]
lv_listings_df = lv_listings_df[lv_listings_df["host_acceptance_rate"].notna()]
lv_listings_df = lv_listings_df[lv_listings_df["last_review"].notna()]
lv_listings_df = lv_listings_df[lv_listings_df["review_scores_rating"].notna()]
lv_listings_df = lv_listings_df[lv_listings_df["description"].notna()]
lv_listings_df = lv_listings_df[lv_listings_df["beds"].notna()]
lv_listings_df = lv_listings_df[lv_listings_df["bedrooms"].notna()]
lv_listings_df = lv_listings_df[lv_listings_df["bathrooms_text"].notna()]

lv_listings_df = lv_listings_df[lv_listings_df["host_about"].apply(is_string)]
lv_listings_df = lv_listings_df[lv_listings_df["name"].apply(is_string)]
lv_listings_df = lv_listings_df[lv_listings_df["description"].apply(is_string)]

lv_listings_df["host_response_rate"] = lv_listings_df["host_response_rate"].apply(percent_to_float)
lv_listings_df["host_acceptance_rate"] = lv_listings_df["host_acceptance_rate"].apply(percent_to_float)
lv_listings_df["price"] = lv_listings_df["price"].apply(dollar_to_float)
lv_listings_df["instant_bookable"] = lv_listings_df["instant_bookable"].apply(str_to_bool)
lv_listings_df["host_identity_verified"] = lv_listings_df["host_identity_verified"].apply(str_to_bool)
lv_listings_df["host_has_profile_pic"] = lv_listings_df["host_has_profile_pic"].apply(str_to_bool)
lv_listings_df["host_is_superhost"] = lv_listings_df["host_is_superhost"].apply(str_to_bool)
lv_listings_df["host_response_time"] = lv_listings_df["host_response_time"].apply(host_response_time_to_int)
lv_listings_df["amenities"] = lv_listings_df["amenities"].apply(amenities_to_list)

lv_listings_df["is_private_room"] = lv_listings_df["room_type"].apply(room_type_to_bool)
lv_listings_df["is_private_bath"] = lv_listings_df["bathrooms_text"].apply(bathrooms_text_to_bool)
lv_listings_df["baths"] = lv_listings_df["bathrooms_text"].apply(bathroom_text_to_float)
lv_listings_df["is_private_overall"] = lv_listings_df["is_private_room"] & lv_listings_df["is_private_bath"]

lv_listings_df["host_since"] = pd.to_datetime(lv_listings_df["host_since"])
lv_listings_df["last_review"] = pd.to_datetime(lv_listings_df["last_review"])

print(lv_listings_df.dtypes)
print(lv_listings_df.shape)
print(lv_listings_df)

lv_calendar_df = pd.read_csv("calendars/lv_calendar.csv")
lv_calendar_df = lv_calendar_df.drop("price", axis=1)
lv_calendar_df = lv_calendar_df.drop("adjusted_price", axis=1)
lv_calendar_df["date"] = pd.to_datetime(lv_calendar_df["date"])
lv_calendar_df["available"] = lv_calendar_df["available"].apply(str_to_bool)

print(lv_calendar_df)
print(lv_calendar_df.dtypes)

id                               float64
name                              object
description                       object
host_since                datetime64[ns]
host_about                        object
host_response_time                 int64
host_response_rate               float64
host_acceptance_rate             float64
host_is_superhost                   bool
host_listings_count              float64
host_has_profile_pic                bool
host_identity_verified              bool
neighbourhood_cleansed            object
room_type                         object
accommodates                       int64
bathrooms_text                    object
bedrooms                         float64
beds                             float64
amenities                         object
price                            float64
minimum_nights                     int64
maximum_nights                     int64
number_of_reviews                  int64
last_review               datetime64[ns]
review_scores_ra

In [72]:
%sql lv_bookings_df << (SELECT listing_id, COUNT(available) AS days_booked FROM lv_calendar_df WHERE available = False GROUP BY listing_id)
%sql lv_combined_df << (SELECT * FROM lv_listings_df LEFT JOIN lv_bookings_df ON lv_listings_df.id = lv_bookings_df.listing_id)
%sql lv_listings_df << SELECT * FROM lv_listings_df ORDER BY price 
%sql lv_combined_df << SELECT * FROM lv_combined_df ORDER BY price 

Returning data to local variable lv_bookings_df
Returning data to local variable lv_combined_df
Returning data to local variable lv_listings_df
Returning data to local variable lv_combined_df


In [73]:
lv_combined_df = lv_combined_df.drop(columns = 'listing_id')
lv_listings_df = lv_listings_df.drop([4933, 4932, 4931, 4930, 4829, 4928, 4927])
lv_combined_df = lv_combined_df.drop([4933, 4932, 4931, 4930, 4829, 4928, 4927])

#### LV Amenity Scraping

In [74]:
lv_amenities_df = pd.DataFrame(lv_combined_df['amenities'])
lv_amenities_list_accum = []
df_len = len(lv_amenities_df.index)
for i in range(0,df_len):
    x = ast.literal_eval(lv_amenities_df.iloc[i, 0])
    lv_amenities_list_accum.append(x)

In [75]:
lv_amenities_frequency={}
for list in lv_amenities_list_accum:
    for item in list:
        if item in lv_amenities_frequency:
            lv_amenities_frequency[item] += 1
        else:
            lv_amenities_frequency[item] = 1

In [76]:
lv_amenities_df = pd.DataFrame(lv_amenities_frequency, index=[0])
lv_amenities_list = lv_amenities_df.columns

In [77]:
lv_amenities_df = pd.melt(lv_amenities_df, value_vars=lv_amenities_list, value_name='lv_count')

In [78]:
%sql lv_amenities_df << SELECT * FROM lv_amenities_df ORDER BY lv_count DESC FETCH FIRST 50 ROWS ONLY;

Returning data to local variable lv_amenities_df


In [79]:
lv_amenities_df = lv_amenities_df.rename(columns={'variable': 'variable_1'})

## Los Angeles Cleaning

In [80]:
# LOS ANGELES CLEANING

la_listings_df = pd.read_csv("listings/la_listings.csv")

la_listings_df = la_listings_df[la_listings_df["host_about"].notna()]
la_listings_df = la_listings_df[la_listings_df["host_response_time"].notna()]
la_listings_df = la_listings_df[la_listings_df["host_response_rate"].notna()]
la_listings_df = la_listings_df[la_listings_df["host_acceptance_rate"].notna()]
la_listings_df = la_listings_df[la_listings_df["last_review"].notna()]
la_listings_df = la_listings_df[la_listings_df["review_scores_rating"].notna()]
la_listings_df = la_listings_df[la_listings_df["description"].notna()]
la_listings_df = la_listings_df[la_listings_df["beds"].notna()]
la_listings_df = la_listings_df[la_listings_df["bedrooms"].notna()]
la_listings_df = la_listings_df[la_listings_df["bathrooms_text"].notna()]

la_listings_df = la_listings_df[la_listings_df["host_about"].apply(is_string)]
la_listings_df = la_listings_df[la_listings_df["name"].apply(is_string)]
la_listings_df = la_listings_df[la_listings_df["description"].apply(is_string)]

la_listings_df["host_response_rate"] = la_listings_df["host_response_rate"].apply(percent_to_float)
la_listings_df["host_acceptance_rate"] = la_listings_df["host_acceptance_rate"].apply(percent_to_float)
la_listings_df["price"] = la_listings_df["price"].apply(dollar_to_float)
la_listings_df["instant_bookable"] = la_listings_df["instant_bookable"].apply(str_to_bool)
la_listings_df["host_identity_verified"] = la_listings_df["host_identity_verified"].apply(str_to_bool)
la_listings_df["host_has_profile_pic"] = la_listings_df["host_has_profile_pic"].apply(str_to_bool)
la_listings_df["host_is_superhost"] = la_listings_df["host_is_superhost"].apply(str_to_bool)
la_listings_df["host_response_time"] = la_listings_df["host_response_time"].apply(host_response_time_to_int)
la_listings_df["amenities"] = la_listings_df["amenities"].apply(amenities_to_list)

la_listings_df["is_private_room"] = la_listings_df["room_type"].apply(room_type_to_bool)
la_listings_df["is_private_bath"] = la_listings_df["bathrooms_text"].apply(bathrooms_text_to_bool)
la_listings_df["baths"] = la_listings_df["bathrooms_text"].apply(bathroom_text_to_float)
la_listings_df["is_private_overall"] = la_listings_df["is_private_room"] & la_listings_df["is_private_bath"]

la_listings_df["host_since"] = pd.to_datetime(la_listings_df["host_since"])
la_listings_df["last_review"] = pd.to_datetime(la_listings_df["last_review"])

print(la_listings_df.dtypes)
print(la_listings_df.shape)
print(la_listings_df)

la_calendar_df = pd.read_csv("calendars/la_calendar.csv")
la_calendar_df = la_calendar_df.drop("price", axis=1)
la_calendar_df = la_calendar_df.drop("adjusted_price", axis=1)
la_calendar_df["date"] = pd.to_datetime(la_calendar_df["date"])
la_calendar_df["available"] = la_calendar_df["available"].apply(str_to_bool)

print(la_calendar_df)
print(la_calendar_df.dtypes)

id                               float64
name                              object
description                       object
host_since                datetime64[ns]
host_about                        object
host_response_time                 int64
host_response_rate               float64
host_acceptance_rate             float64
host_is_superhost                   bool
host_listings_count              float64
host_has_profile_pic                bool
host_identity_verified              bool
neighbourhood_cleansed            object
room_type                         object
accommodates                       int64
bathrooms_text                    object
bedrooms                         float64
beds                             float64
amenities                         object
price                            float64
minimum_nights                     int64
maximum_nights                     int64
number_of_reviews                  int64
last_review               datetime64[ns]
review_scores_ra

In [81]:
%sql la_bookings_df << (SELECT listing_id, COUNT(available) AS days_booked FROM la_calendar_df WHERE available = False GROUP BY listing_id)
%sql la_combined_df << (SELECT * FROM la_listings_df LEFT JOIN la_bookings_df ON la_listings_df.id = la_bookings_df.listing_id)
%sql la_listings_df << SELECT * FROM la_listings_df ORDER BY price 
%sql la_combined_df << SELECT * FROM la_combined_df ORDER BY price 

Returning data to local variable la_bookings_df
Returning data to local variable la_combined_df
Returning data to local variable la_listings_df
Returning data to local variable la_combined_df


In [82]:
la_combined_df = la_combined_df.drop(columns = 'listing_id')
la_listings_df = la_listings_df.drop([15753])
la_combined_df = la_combined_df.drop([15753])

#### LA Amenity Scraping

In [83]:
la_amenities_df = pd.DataFrame(la_combined_df['amenities'])
la_amenities_list_accum = []
df_len = len(la_amenities_df.index)
for i in range(0,df_len):
    x = ast.literal_eval(la_amenities_df.iloc[i, 0])
    la_amenities_list_accum.append(x)

In [84]:
la_amenities_frequency={}
for list in la_amenities_list_accum:
    for item in list:
        if item in la_amenities_frequency:
            la_amenities_frequency[item] += 1
        else:
            la_amenities_frequency[item] = 1

In [85]:
la_amenities_df = pd.DataFrame(la_amenities_frequency, index=[0])
la_amenities_list = la_amenities_df.columns

In [86]:
la_amenities_df = pd.melt(la_amenities_df, value_vars=la_amenities_list, value_name='la_count')

In [87]:
%sql la_amenities_df << SELECT * FROM la_amenities_df ORDER BY la_count DESC FETCH FIRST 50 ROWS ONLY;

Returning data to local variable la_amenities_df


In [88]:
la_amenities_df = la_amenities_df.rename(columns={'variable': 'variable_1'})

## Nashville Cleaning

In [89]:
# NASHVILLE CLEANING

nsh_listings_df = pd.read_csv("listings/nsh_listings.csv")

nsh_listings_df = nsh_listings_df[nsh_listings_df["host_about"].notna()]
nsh_listings_df = nsh_listings_df[nsh_listings_df["host_response_time"].notna()]
nsh_listings_df = nsh_listings_df[nsh_listings_df["host_response_rate"].notna()]
nsh_listings_df = nsh_listings_df[nsh_listings_df["host_acceptance_rate"].notna()]
nsh_listings_df = nsh_listings_df[nsh_listings_df["last_review"].notna()]
nsh_listings_df = nsh_listings_df[nsh_listings_df["review_scores_rating"].notna()]
nsh_listings_df = nsh_listings_df[nsh_listings_df["description"].notna()]
nsh_listings_df = nsh_listings_df[nsh_listings_df["beds"].notna()]
nsh_listings_df = nsh_listings_df[nsh_listings_df["bedrooms"].notna()]
nsh_listings_df = nsh_listings_df[nsh_listings_df["bathrooms_text"].notna()]

nsh_listings_df = nsh_listings_df[nsh_listings_df["host_about"].apply(is_string)]
nsh_listings_df = nsh_listings_df[nsh_listings_df["name"].apply(is_string)]
nsh_listings_df = nsh_listings_df[nsh_listings_df["description"].apply(is_string)]

nsh_listings_df["host_response_rate"] = nsh_listings_df["host_response_rate"].apply(percent_to_float)
nsh_listings_df["host_acceptance_rate"] = nsh_listings_df["host_acceptance_rate"].apply(percent_to_float)
nsh_listings_df["price"] = nsh_listings_df["price"].apply(dollar_to_float)
nsh_listings_df["instant_bookable"] = nsh_listings_df["instant_bookable"].apply(str_to_bool)
nsh_listings_df["host_identity_verified"] = nsh_listings_df["host_identity_verified"].apply(str_to_bool)
nsh_listings_df["host_has_profile_pic"] = nsh_listings_df["host_has_profile_pic"].apply(str_to_bool)
nsh_listings_df["host_is_superhost"] = nsh_listings_df["host_is_superhost"].apply(str_to_bool)
nsh_listings_df["host_response_time"] = nsh_listings_df["host_response_time"].apply(host_response_time_to_int)
nsh_listings_df["amenities"] = nsh_listings_df["amenities"].apply(amenities_to_list)

nsh_listings_df["is_private_room"] = nsh_listings_df["room_type"].apply(room_type_to_bool)
nsh_listings_df["is_private_bath"] = nsh_listings_df["bathrooms_text"].apply(bathrooms_text_to_bool)
nsh_listings_df["baths"] = nsh_listings_df["bathrooms_text"].apply(bathroom_text_to_float)
nsh_listings_df["is_private_overall"] = nsh_listings_df["is_private_room"] & nsh_listings_df["is_private_bath"]

nsh_listings_df["host_since"] = pd.to_datetime(nsh_listings_df["host_since"])
nsh_listings_df["last_review"] = pd.to_datetime(nsh_listings_df["last_review"])

print(nsh_listings_df.dtypes)
print(nsh_listings_df.shape)
print(nsh_listings_df)

nsh_calendar_df = pd.read_csv("calendars/nsh_calendar.csv")
nsh_calendar_df = nsh_calendar_df.drop("price", axis=1)
nsh_calendar_df = nsh_calendar_df.drop("adjusted_price", axis=1)
nsh_calendar_df["date"] = pd.to_datetime(nsh_calendar_df["date"])
nsh_calendar_df["available"] = nsh_calendar_df["available"].apply(str_to_bool)

print(nsh_calendar_df)
print(nsh_calendar_df.dtypes)

id                                 int64
name                              object
description                       object
host_since                datetime64[ns]
host_about                        object
host_response_time                 int64
host_response_rate               float64
host_acceptance_rate             float64
host_is_superhost                   bool
host_listings_count              float64
host_has_profile_pic                bool
host_identity_verified              bool
neighbourhood_cleansed            object
room_type                         object
accommodates                       int64
bathrooms_text                    object
bedrooms                         float64
beds                             float64
amenities                         object
price                            float64
minimum_nights                     int64
maximum_nights                     int64
number_of_reviews                  int64
last_review               datetime64[ns]
review_scores_ra

In [90]:
%sql nsh_bookings_df << (SELECT listing_id, COUNT(available) AS days_booked FROM nsh_calendar_df WHERE available = False GROUP BY listing_id)
%sql nsh_combined_df << (SELECT * FROM nsh_listings_df LEFT JOIN nsh_bookings_df ON nsh_listings_df.id = nsh_bookings_df.listing_id)
%sql nsh_listings_df << SELECT * FROM nsh_listings_df ORDER BY price 
%sql nsh_combined_df << SELECT * FROM nsh_combined_df ORDER BY price 

Returning data to local variable nsh_bookings_df
Returning data to local variable nsh_combined_df
Returning data to local variable nsh_listings_df
Returning data to local variable nsh_combined_df


In [91]:
nsh_combined_df = nsh_combined_df.drop(columns = 'listing_id')
nsh_listings_df = nsh_listings_df.drop([4260])
nsh_combined_df = nsh_combined_df.drop([4260])

#### NASH Amenity Scraping

In [92]:
nsh_amenities_df = pd.DataFrame(nsh_combined_df['amenities'])
nsh_amenities_list_accum = []
df_len = len(nsh_amenities_df.index)
for i in range(0,df_len):
    x = ast.literal_eval(nsh_amenities_df.iloc[i, 0])
    nsh_amenities_list_accum.append(x)

In [93]:
nsh_amenities_frequency={}
for list in nsh_amenities_list_accum:
    for item in list:
        if item in nsh_amenities_frequency:
            nsh_amenities_frequency[item] += 1
        else:
            nsh_amenities_frequency[item] = 1

In [94]:
nsh_amenities_df = pd.DataFrame(nsh_amenities_frequency, index=[0])
nsh_amenities_list = nsh_amenities_df.columns

In [95]:
nsh_amenities_df = pd.melt(nsh_amenities_df, value_vars=nsh_amenities_list, value_name='nsh_count')

In [96]:
%sql nsh_amenities_df << SELECT * FROM nsh_amenities_df ORDER BY nsh_count DESC FETCH FIRST 50 ROWS ONLY;

Returning data to local variable nsh_amenities_df


In [97]:
nsh_amenities_df = nsh_amenities_df.rename(columns={'variable': 'variable_1'})

## Twin Cities Cleaning

In [98]:
# TWIN CITIES CLEANING

msp_listings_df = pd.read_csv("listings/msp_listings.csv")

msp_listings_df = msp_listings_df[msp_listings_df["host_about"].notna()]
msp_listings_df = msp_listings_df[msp_listings_df["host_response_time"].notna()]
msp_listings_df = msp_listings_df[msp_listings_df["host_response_rate"].notna()]
msp_listings_df = msp_listings_df[msp_listings_df["host_acceptance_rate"].notna()]
msp_listings_df = msp_listings_df[msp_listings_df["last_review"].notna()]
msp_listings_df = msp_listings_df[msp_listings_df["review_scores_rating"].notna()]
msp_listings_df = msp_listings_df[msp_listings_df["description"].notna()]
msp_listings_df = msp_listings_df[msp_listings_df["beds"].notna()]
msp_listings_df = msp_listings_df[msp_listings_df["bedrooms"].notna()]
msp_listings_df = msp_listings_df[msp_listings_df["bathrooms_text"].notna()]

msp_listings_df = msp_listings_df[msp_listings_df["host_about"].apply(is_string)]
msp_listings_df = msp_listings_df[msp_listings_df["name"].apply(is_string)]
msp_listings_df = msp_listings_df[msp_listings_df["description"].apply(is_string)]

msp_listings_df["host_response_rate"] = msp_listings_df["host_response_rate"].apply(percent_to_float)
msp_listings_df["host_acceptance_rate"] = msp_listings_df["host_acceptance_rate"].apply(percent_to_float)
msp_listings_df["price"] = msp_listings_df["price"].apply(dollar_to_float)
msp_listings_df["instant_bookable"] = msp_listings_df["instant_bookable"].apply(str_to_bool)
msp_listings_df["host_identity_verified"] = msp_listings_df["host_identity_verified"].apply(str_to_bool)
msp_listings_df["host_has_profile_pic"] = msp_listings_df["host_has_profile_pic"].apply(str_to_bool)
msp_listings_df["host_is_superhost"] = msp_listings_df["host_is_superhost"].apply(str_to_bool)
msp_listings_df["host_response_time"] = msp_listings_df["host_response_time"].apply(host_response_time_to_int)
msp_listings_df["amenities"] = msp_listings_df["amenities"].apply(amenities_to_list)

msp_listings_df["is_private_room"] = msp_listings_df["room_type"].apply(room_type_to_bool)
msp_listings_df["is_private_bath"] = msp_listings_df["bathrooms_text"].apply(bathrooms_text_to_bool)
msp_listings_df["baths"] = msp_listings_df["bathrooms_text"].apply(bathroom_text_to_float)
msp_listings_df["is_private_overall"] = msp_listings_df["is_private_room"] & msp_listings_df["is_private_bath"]

msp_listings_df["host_since"] = pd.to_datetime(msp_listings_df["host_since"])
msp_listings_df["last_review"] = pd.to_datetime(msp_listings_df["last_review"])

print(msp_listings_df.dtypes)
print(msp_listings_df.shape)
print(msp_listings_df)

msp_calendar_df = pd.read_csv("calendars/msp_calendar.csv")
msp_calendar_df = msp_calendar_df.drop("price", axis=1)
msp_calendar_df = msp_calendar_df.drop("adjusted_price", axis=1)
msp_calendar_df["date"] = pd.to_datetime(msp_calendar_df["date"])
msp_calendar_df["available"] = msp_calendar_df["available"].apply(str_to_bool)

print(msp_calendar_df)
print(msp_calendar_df.dtypes)

id                               float64
name                              object
description                       object
host_since                datetime64[ns]
host_about                        object
host_response_time                 int64
host_response_rate               float64
host_acceptance_rate             float64
host_is_superhost                   bool
host_listings_count                int64
host_has_profile_pic                bool
host_identity_verified              bool
neighbourhood_cleansed            object
room_type                         object
accommodates                       int64
bathrooms_text                    object
bedrooms                         float64
beds                             float64
amenities                         object
price                            float64
minimum_nights                     int64
maximum_nights                     int64
number_of_reviews                  int64
last_review               datetime64[ns]
review_scores_ra

In [99]:
%sql msp_bookings_df << (SELECT listing_id, COUNT(available) AS days_booked FROM msp_calendar_df WHERE available = False GROUP BY listing_id)
%sql msp_combined_df << (SELECT * FROM msp_listings_df LEFT JOIN msp_bookings_df ON msp_listings_df.id = msp_bookings_df.listing_id)
%sql msp_listings_df << SELECT * FROM msp_listings_df ORDER BY price 
%sql msp_combined_df << SELECT * FROM msp_combined_df ORDER BY price 

Returning data to local variable msp_bookings_df
Returning data to local variable msp_combined_df
Returning data to local variable msp_listings_df
Returning data to local variable msp_combined_df


In [100]:
msp_combined_df = msp_combined_df.drop(columns = 'listing_id')
msp_listings_df = msp_listings_df.drop([1771])
msp_combined_df = msp_combined_df.drop([1771])

#### MSP Amenity Scraping

In [101]:
msp_amenities_df = pd.DataFrame(msp_combined_df['amenities'])
msp_amenities_list_accum = []
df_len = len(msp_amenities_df.index)
for i in range(0,df_len):
    x = ast.literal_eval(msp_amenities_df.iloc[i, 0])
    msp_amenities_list_accum.append(x)

In [102]:
msp_amenities_frequency={}
for list in msp_amenities_list_accum:
    for item in list:
        if item in msp_amenities_frequency:
            msp_amenities_frequency[item] += 1
        else:
            msp_amenities_frequency[item] = 1

In [103]:
msp_amenities_df = pd.DataFrame(msp_amenities_frequency, index=[0])
msp_amenities_list = msp_amenities_df.columns

In [104]:
msp_amenities_df = pd.melt(msp_amenities_df, value_vars=msp_amenities_list, value_name='msp_count')

In [105]:
%sql msp_amenities_df << SELECT * FROM msp_amenities_df ORDER BY msp_count DESC FETCH FIRST 50 ROWS ONLY;

Returning data to local variable msp_amenities_df


In [106]:
msp_amenities_df = msp_amenities_df.rename(columns={'variable': 'variable_1'})

In [107]:
%sql amenities_df << SELECT * FROM nyc_amenities_df INNER JOIN bos_amenities_df ON nyc_amenities_df.variable = bos_amenities_df.variable_1;
%sql amenities_df << SELECT * FROM amenities_df INNER JOIN chi_amenities_df ON amenities_df.variable = chi_amenities_df.variable_1;
%sql amenities_df << SELECT * FROM amenities_df INNER JOIN dc_amenities_df ON amenities_df.variable = dc_amenities_df.variable_1;
%sql amenities_df << SELECT * FROM amenities_df INNER JOIN dal_amenities_df ON amenities_df.variable = dal_amenities_df.variable_1;
%sql amenities_df << SELECT * FROM amenities_df INNER JOIN lv_amenities_df ON amenities_df.variable = lv_amenities_df.variable_1;
%sql amenities_df << SELECT * FROM amenities_df INNER JOIN la_amenities_df ON amenities_df.variable = la_amenities_df.variable_1;
%sql amenities_df << SELECT * FROM amenities_df INNER JOIN nsh_amenities_df ON amenities_df.variable = nsh_amenities_df.variable_1;
%sql amenities_df << SELECT * FROM amenities_df INNER JOIN msp_amenities_df ON amenities_df.variable = msp_amenities_df.variable_1;

Returning data to local variable amenities_df
Returning data to local variable amenities_df
Returning data to local variable amenities_df
Returning data to local variable amenities_df
Returning data to local variable amenities_df
Returning data to local variable amenities_df
Returning data to local variable amenities_df
Returning data to local variable amenities_df


In [108]:
amenities_df = amenities_df.drop(columns=['variable_1','variable_1_1', 'variable_1_2', 'variable_1_3', 'variable_1_4', 'variable_1_5', 'variable_1_6'])

In [109]:
# amenities_df = amenities_df.drop([0,1,2,3,4,5,7,8,10,11, 12, 13, 14, 15, 16, 17, 19, 22, 23, 26, 34, 38, 39], axis=0)
keep = ['6', '19', '25', '28', '30', '35']
for i in range(len(amenities_df)):
    if str(i) not in keep:
        amenities_df=amenities_df.drop([i], axis=0)

print(amenities_df)

            variable  nyc_count  bos_count  chi_count  dc_count  dal_count  \
6       Coffee maker       7421       1604       2950      2083       2383   
19           Heating       8393       1653       2533      1986       1910   
25        Dishwasher       3259       1101       1998      1476       2026   
28  Air conditioning       6860       1191       1861      1526       1574   
30           Toaster       3046        634       1190       730       1201   
35      Wine glasses       3285        621       1165       798       1201   

    lv_count  la_count  nsh_count  msp_count  
6       3974     12749       3976       1585  
19      3640     11703       3202       1288  
25      3217      8258       3223       1022  
28      3189      8247       2780        921  
30      1821      5169       1399        822  
35      1480      4995       1438        732  


## Export Cleaned Data

In [113]:
# EXPORT CLEANED DATA INTO CSV
nyc_listings_df.to_csv("./cleaned_data/nyc_listings_clean.csv", encoding="utf-8")
nyc_combined_df.to_csv("./cleaned_data/nyc_combined_clean.csv", encoding="utf-8")
nyc_calendar_df.to_csv("./cleaned_data/nyc_calendar_clean.csv", encoding="utf-8")

bos_listings_df.to_csv("./cleaned_data/bos_listings_clean.csv", encoding="utf-8")
bos_combined_df.to_csv("./cleaned_data/bos_combined_clean.csv", encoding="utf-8")
bos_calendar_df.to_csv("./cleaned_data/bos_calendar_clean.csv", encoding="utf-8")

nsh_listings_df.to_csv("./cleaned_data/nsh_listings_clean.csv", encoding="utf-8")
nsh_combined_df.to_csv("./cleaned_data/nsh_combined_clean.csv", encoding="utf-8")
nsh_calendar_df.to_csv("./cleaned_data/nsh_calendar_clean.csv", encoding="utf-8")

dal_listings_df.to_csv("./cleaned_data/dal_listings_clean.csv", encoding="utf-8")
dal_combined_df.to_csv("./cleaned_data/dal_combined_clean.csv", encoding="utf-8")
dal_calendar_df.to_csv("./cleaned_data/dal_calendar_clean.csv", encoding="utf-8")

dc_listings_df.to_csv("./cleaned_data/dc_listings_clean.csv", encoding="utf-8")
dc_combined_df.to_csv("./cleaned_data/dc_combined_clean.csv", encoding="utf-8")
dc_calendar_df.to_csv("./cleaned_data/dc_calendar_clean.csv", encoding="utf-8")

chi_listings_df.to_csv("./cleaned_data/chi_listings_clean.csv", encoding="utf-8")
chi_combined_df.to_csv("./cleaned_data/chi_combined_clean.csv", encoding="utf-8")
chi_calendar_df.to_csv("./cleaned_data/chi_calendar_clean.csv", encoding="utf-8")
chi_amenities_df.to_csv("./cleaned_data/chi_amenities_clean.csv", encoding="utf-8")

la_listings_df.to_csv("./cleaned_data/la_listings_clean.csv", encoding="utf-8")
la_combined_df.to_csv("./cleaned_data/la_combined_clean.csv", encoding="utf-8")
la_calendar_df.to_csv("./cleaned_data/la_calendar_clean.csv", encoding="utf-8")

lv_listings_df.to_csv("./cleaned_data/lv_listings_clean.csv", encoding="utf-8")
lv_combined_df.to_csv("./cleaned_data/lv_combined_clean.csv", encoding="utf-8")
lv_calendar_df.to_csv("./cleaned_data/lv_calendar_clean.csv", encoding="utf-8")

msp_listings_df.to_csv("./cleaned_data/msp_listings_clean.csv", encoding="utf-8")
msp_combined_df.to_csv("./cleaned_data/msp_combined_clean.csv", encoding="utf-8")
msp_calendar_df.to_csv("./cleaned_data/msp_calendar_clean.csv", encoding="utf-8")


In [114]:
# Export amenities
amenities_df.to_csv("./cleaned_data/all_amenities_clean.csv", encoding="utf-8")